# Demo of the Adaptive Ground Point Filtering Library

This notebook gives a tour of the capabilities of `filteradapt`. It is continuously updated while we develop the library. You can execute the code in each cell by pressing `Ctrl+Enter`. To start your own project based on `filteradapt`, you can either edit this notebook or start a new one.

The first thing to do in a Jupyter notebook that uses `filteradapt` is to import the library:

In [ ]:
import filteradapt